In [ ]:
# Magic functions -- Run Once
%load_ext autoreload
%autoreload 2
%matplotlib notebook

# Move up one folder to reach the repo root
%cd ..

from utils.notebook.generic import full_width_notebook

full_width_notebook()

In [ ]:
# Paths, Imports & Configs
import pandas as pd
from copy import deepcopy
from utils.notebook.experiment_explorer import get_experiments

from utils.notebook.pandas import sub_cols_with_cond_and_create_new_col, grouped_scatter

root_data_path = "data"
#root_output_path = "output/training"
root_output_path = "output/synced_training"

# Helpers column selectors
run_info_columns = ['prefix', 'nb_sample', 'config', 'nb_epoch_runned', 'stopped_early']
more_accuracy_info_columns = ['0.6_at_epoch', '0.7_at_epoch', '0.8_at_epoch', '0.9_at_epoch']
accuracy_columns = ['best_val_acc', 'train_acc', 'test_acc']
loss_columns = ['best_val_loss', 'train_loss', 'test_loss']
all_training_stats = ['all_train_acc', 'all_train_loss', 'all_val_acc', 'all_val_loss']
model_config_columns = ['nb_trainable_param', 'word_embedding_dim', 'rnn_state_size', 'extractor_type', 
                        'stem_out_chan', 'nb_resblock', 'resblocks_out_chan', 'classifier_conv_out_chan', 
                        'classifier_type', 'classifier_global_pool', 'nb_answer']
optimizer_columns = ['optimizer_type', 'optimizer_lr', 'optimizer_weight_decay', 'dropout_drop_prob']
preprocessing_columns = ['pad_to_largest', 'resized_height', 'resized_width']
timing_columns = ['train_time', 'mean_epoch_time']
other_columns = ['nb_scene', 'nb_q_per_scene', 'date', 'batch_size', 'resnet_features', 'test_version', 
                 'random_seed', 'nb_non_trainable_param', 'total_nb_param', 'nb_epoch', 'stop_accuracy', 'folder', 'gpu_name']

# Retrieve all experimentsi infos
experiments = get_experiments(root_output_path)

# Pretty printing
format_dict = {
    'total_nb_param': "{:,d}".format,
    'nb_non_trainable_param': "{:,d}".format,
    'nb_trainable_param': "{:,d}".format,
    'nb_trainable_param_round': "~ {:,d}".format,
    'nb_sample': "{:,d}".format,
    'nb_scene': "{:,d}".format,
    'rnn_state_size': "{:,d}".format,
    'optimizer_lr': "{:.2e}".format,
    'optimizer_weight_decay': "{:.2e}".format,
    'best_val_acc': lambda x: "{:.2f}%".format(x*100) if not pd.isnull(x) else None,
    'train_acc': lambda x: "{:.2f}%".format(x*100) if not pd.isnull(x) else None,
    'test_acc': lambda x: "{:.2f}%".format(x*100) if not pd.isnull(x) else None,
    '0.6_at_epoch': lambda x: x if not pd.isnull(x) else None,
    '0.7_at_epoch': lambda x: x if not pd.isnull(x) else None,
    '0.8_at_epoch': lambda x: x if not pd.isnull(x) else None,
    '0.9_at_epoch': lambda x: x if not pd.isnull(x) else None
}

latex_format_dict = deepcopy(format_dict)
latex_format_dict['nb_sample'] = lambda x: "{:d}k".format(x//1000)
latex_format_dict['nb_scene'] = lambda x: "{:d}k".format(x//1000)
latex_format_dict['best_val_acc'] = lambda x: "{:.2f}".format(x*100) if not pd.isnull(x) else None
latex_format_dict['train_acc'] = lambda x: "{:.2f}".format(x*100) if not pd.isnull(x) else None
latex_format_dict['test_acc'] = lambda x: "{:.2f}".format(x*100) if not pd.isnull(x) else None

# Round number params to the closest thousand to facilitate comparison
experiments['nb_trainable_param_round'] = experiments['nb_trainable_param'].apply(lambda x: x//1000 * 1000)

pd.set_option('display.max_colwidth', -1)

exp_results_cols = run_info_columns + ['nb_scene', 'nb_q_per_scene'] + accuracy_columns + more_accuracy_info_columns + ['nb_trainable_param_round', 'gpu_name']
exp_results = experiments[exp_results_cols].sort_values(['nb_scene', 'test_acc'], ascending=[True, False])

experiments.columns.values


In [ ]:
print("Available configs :")
set(experiments['config'].values)

In [ ]:
exp_results[(exp_results['stopped_early'] == 'stop_threshold') & (~exp_results['config'].str.contains('extractor'))]

In [ ]:
#exp_results[exp_results['nb_scene'] == 50000]
#exp_results[exp_results['nb_sample'] == 200000].sort_values(['nb_scene', 'nb_q_per_scene']).style.format(format_dict)
exp_results

In [ ]:
exp_results[(exp_results['nb_scene'] <= 10000) & (exp_results['config'].str.contains('reduction'))].sort_values('test_acc').style.format(format_dict)

In [ ]:
df_filter = ((experiments['config'].str.contains('reduction')) & (~experiments['config'].str.contains('stem')) & (~experiments['config'].str.contains('extractor')) & (~experiments['config'].str.contains('resblock')) & (~experiments['config'].str.contains('avg'))) | ((experiments['config'] == 'film_original') & (exp_results['nb_scene'] == 20000) & (exp_results['nb_q_per_scene'] == 20))
reduction_experiments = experiments[experiments['config'].str.contains('reduction')][exp_results_cols + timing_columns].sort_values('config')
reduction_experiments#[reduction_experiments['stopped_early'] == 'NO']['folder'].values

In [ ]:
exp_results[(exp_results['config'].str.contains('interlaced_smallest'))].sort_values('test_acc', ascending=True).style.format(format_dict)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np

def plot_training_stats(dataframe, names=None, title=None, colormap=cm.viridis, ax=None):
    nb_rows = len(dataframe.index)

    colorlist = {key: colors.rgb2hex(colormap(i)) for key, i in
                 zip(dataframe.index, np.linspace(0, 0.9, nb_rows))}

    if title is None:
        title = "Training Stats"
    
    if ax is None:
        fig, axs = plt.subplots(2,1)
        
    fig.suptitle(title)
    axs[0].set_title("Accuracy")
    axs[1].set_title("Loss")
        
    for row in dataframe.itertuples():
        label = f"{row.config} - {row.nb_scene//1000}k_{row.nb_q_per_scene}_q"
        lines = axs[0].plot(row.all_train_acc, label=f"TRAIN - {label}")
        axs[0].plot(row.all_val_acc, color=lines[0].get_color(), linestyle="--")
        
        axs[1].plot(row.all_train_loss, color=lines[0].get_color())
        axs[1].plot(row.all_val_loss, label=f"VAL - {label}", color=lines[0].get_color(), linestyle="--")
        
    # Draw legend out of the plot
    for ax in axs:
        box = ax.get_position()
        ax.set_position([box.x0, box.y0, box.width * 0.5, box.height])

        ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

    fig.tight_layout()
    
    return fig, ax

    #for key, group in dataframe.groupby(group_key):
    #    group.plot.scatter(ax=ax, x=x_axis, y=y_axis, c=colorlist[key], label=key, title=title)
    
with_training_stats = experiments[exp_results_cols + all_training_stats]
fig, ax = plot_training_stats(with_training_stats[(with_training_stats['config'].str.contains('separated'))])

In [ ]:
plot_training_stats(with_training_stats[with_training_stats['nb_q_per_scene'] == 1])

In [ ]:
# Visualization of test_acc by (nb_sample, nb_scene)
exp_results[(~exp_results['config'].str.contains('film_original')) & (~exp_results['config'].str.contains('extractor'))].hist(column='train_acc', by=['nb_sample', 'nb_scene'])


In [ ]:


grouped_scatter(experiments, 'config', 'nb_sample', 'test_acc')

#grouped_scatter(experiments[experiments['config'] == 'film_original'], 'nb_q_per_scene', 'nb_scene', 'test_acc')

test = experiments.copy()

test['perf_delta_orig_vs_small_rnn'] = test[test['config'] == 'film_original']['test_acc'] - test[test['config'] == 'film_original_small_rnn']['test_acc']

In [ ]:
test = sub_cols_with_cond_and_create_new_col(experiments.copy(), 'perf_delta_orig_vs_small_rnn', 'test_acc', test['config'] == 'film_original_small_rnn', test['config'] == 'film_original', test['config'] == 'film_original_small_rnn')

test.sort_values(['nb_sample', 'nb_scene'], ascending=False)[run_info_columns + ['test_acc', 'perf_delta_orig_vs_small_rnn']]

In [ ]:
latex_table = experiments[experiments['config'] == 'film_original'][['nb_sample', 'nb_scene', 'nb_q_per_scene', 'train_acc', 'best_val_acc', 'test_acc', '0.8_at_epoch']].sort_values(['nb_sample', 'nb_scene']).fillna("None").to_latex(formatters=latex_format_dict)
print(latex_table)

# Table Generation

In [ ]:
## Network reduction -- GRU units
df_filter = ((experiments['config'].str.contains('reduction')) & (~experiments['config'].str.contains('stem')) & (~experiments['config'].str.contains('extractor')) & (~experiments['config'].str.contains('resblock')) & (~experiments['config'].str.contains('avg'))) | ((experiments['config'] == 'film_original') & (exp_results['nb_scene'] == 20000) & (exp_results['nb_q_per_scene'] == 20))
columns = ['rnn_state_size', 'nb_trainable_param', 'train_acc', 'best_val_acc', 'test_acc', 'nb_epoch_runned', 'train_time', 'random_seed']
reduction_experiments = experiments[df_filter][columns].sort_values('nb_trainable_param', ascending=False)
#print(reduction_experiments.to_latex(index=False, formatters=latex_format_dict))
reduction_experiments.style.format(latex_format_dict)

In [ ]:
## Network reduction -- 1x1 Conv classifier
df_filter = ((experiments['config'].str.contains('reduction')) & (~experiments['config'].str.contains('stem')) & (~experiments['config'].str.contains('extractor')) & (~experiments['config'].str.contains('resblock')) & (experiments['config'].str.contains('avg'))) | ((experiments['config'] == 'film_original_avg') & (exp_results['nb_scene'] == 20000) & (exp_results['nb_q_per_scene'] == 20))
columns = ['rnn_state_size', 'nb_trainable_param', 'train_acc', 'best_val_acc', 'test_acc', 'nb_epoch_runned', 'train_time']
reduction_experiments = experiments[df_filter][columns].sort_values('nb_trainable_param', ascending=False)
#print(reduction_experiments.to_latex(index=False, formatters=latex_format_dict))
reduction_experiments.style.format(latex_format_dict)

In [ ]:
## Network reduction -- Extractor 64
#reduction_original_rnn_256_avg_extractor_64
#reduction_original_rnn_256_avg_extractor_64_stem_64
#reduction_original_rnn_256_avg_extractor_64_stem_64_1_resblock
#reduction_original_rnn_256_avg_extractor_64_stem_64_2_resblock
#reduction_original_rnn_256_avg_extractor_64_stem_64_3_resblock

df_filter = experiments['config'].str.contains('reduction_original_rnn_256_avg_extractor_64')
columns = ['nb_resblock', 'extractor_features', 'stem_out_chan', 'nb_trainable_param', 'train_acc', 'best_val_acc', 'test_acc', 'nb_epoch_runned', 'train_time']
reduction_experiments = experiments[df_filter]
reduction_experiments.loc[:, 'extractor_features'] = 64

reduction_experiments = reduction_experiments[columns].sort_values('nb_trainable_param', ascending=False)
#print(reduction_experiments.to_latex(index=False, formatters=latex_format_dict))
reduction_experiments.style.format(latex_format_dict)

In [ ]:
## Network reduction -- Extractor 32
#reduction_original_rnn_256_avg_extractor_32
#reduction_original_rnn_256_avg_extractor_32_stem_32
#reduction_original_rnn_256_avg_extractor_32_stem_32_1_resblock
#reduction_original_rnn_256_avg_extractor_32_stem_32_2_resblock
#reduction_original_rnn_256_avg_extractor_32_stem_32_3_resblock

df_filter = (experiments['config'].str.contains('reduction_original_rnn_256_avg_extractor_32')) & (experiments['nb_epoch_runned'] <= 40)
columns = ['nb_resblock', 'extractor_features', 'stem_out_chan', 'nb_trainable_param', 'train_acc', 'best_val_acc', 'test_acc', 'nb_epoch_runned', 'train_time']
reduction_experiments = experiments[df_filter]
reduction_experiments.loc[:, 'extractor_features'] = 32

reduction_experiments = reduction_experiments[columns].sort_values('nb_trainable_param', ascending=False)
#print(reduction_experiments.to_latex(index=False, formatters=latex_format_dict))
reduction_experiments.style.format(latex_format_dict)

In [ ]:
## Network reduction -- Combined Extractor 32 & 64
#reduction_original_rnn_256_avg_extractor_64
#reduction_original_rnn_256_avg_extractor_64_stem_64
#reduction_original_rnn_256_avg_extractor_64_stem_64_1_resblock
#reduction_original_rnn_256_avg_extractor_64_stem_64_2_resblock
#reduction_original_rnn_256_avg_extractor_64_stem_64_3_resblock

df_64_filter = experiments['config'].str.contains('reduction_original_rnn_256_avg_extractor_64')
df_32_filter = (experiments['config'].str.contains('reduction_original_rnn_256_avg_extractor_32')) & (experiments['nb_epoch_runned'] <= 40)
columns = ['rnn_state_size', 'nb_resblock', 'extractor_features', 'stem_out_chan', 'nb_trainable_param', 'train_acc', 'best_val_acc', 'test_acc', 'nb_epoch_runned', 'train_time']
reduction_experiments = experiments[df_64_filter | df_32_filter]
reduction_experiments.loc[df_64_filter, 'extractor_features'] = 64
reduction_experiments.loc[df_32_filter, 'extractor_features'] = 32
reduction_experiments['train_time'] = reduction_experiments['train_time'].apply(lambda x: x._repr_base(format="sub_day").split('.')[0])

reduction_experiments = reduction_experiments[columns].sort_values(['extractor_features', 'stem_out_chan', 'nb_resblock'], ascending=[False, False, False])
print(reduction_experiments.to_latex(index=False, formatters=latex_format_dict))
#reduction_experiments.style.format(latex_format_dict)